# Purpose

Work on early routing code to enable (primarily) some useful screenshots for the design prototype but also to test out some theories.

# Imports

In [1]:

%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import pandas as pd

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-2.0")
logger.info("TEST!")

2024-07-29_T20_01_11EDT: INFO (Notebook-2.0:L16) - TEST!


In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Trying out Folium

I hear it's a very capable high-quality mapping tool, so let's give it a whirl! We'll do everything initially with OpenStreetMap and open source routing to get a feel for things (may eventually switch to Google Routes API).

## Starting Point Map

In [161]:
help(single_point_map)

Help on Map in module folium.folium object:

class Map(folium.elements.JSCSSMixin, folium.map.Evented)
 |  Map(location: Optional[Sequence[float]] = None, width: Union[str, float] = '100%', height: Union[str, float] = '100%', left: Union[str, float] = '0%', top: Union[str, float] = '0%', position: str = 'relative', tiles: Union[str, folium.raster_layers.TileLayer, NoneType] = 'OpenStreetMap', attr: Optional[str] = None, min_zoom: Optional[int] = None, max_zoom: Optional[int] = None, zoom_start: int = 10, min_lat: float = -90, max_lat: float = 90, min_lon: float = -180, max_lon: float = 180, max_bounds: bool = False, crs: str = 'EPSG3857', control_scale: bool = False, prefer_canvas: bool = False, no_touch: bool = False, disable_3d: bool = False, png_enabled: bool = False, zoom_control: Union[bool, str] = True, font_size: str = '1rem', **kwargs: Union[str, float, bool, Sequence, dict, NoneType])
 |  
 |  Create a Map with Folium and Leaflet.js
 |  
 |  Generate a base map of given width 

In [195]:
from evlens.visualization.folium_tools import get_single_point
import folium

WHITEHOUSE_COORDS = (38.8964457,-77.035921) # for funsies
BOSTON_COORDS = (42.3144474,-71.0526845)
CLEVELAND_COORDS = (41.4973681,-82.0176764)

single_point_map = get_single_point(WHITEHOUSE_COORDS, 'White House', include_zoom_widget=False)
single_point_map

## Visualizing a Route

In [4]:
import openrouteservice as ors
import os

client = ors.Client(key=os.getenv('ORS_API_KEY'))

In [131]:
type(route)

dict

In [196]:
from evlens.data.routing import get_openrouting_route

r = get_openrouting_route(WHITEHOUSE_COORDS, CLEVELAND_COORDS, reverse_coordinates=True)
len(r)

4324

In [199]:
# Add our destination point
start_icon = folium.Icon(color='green', icon='fa-solid fa-play', prefix='fa')
end_icon = folium.Icon(color='red', icon='fa-solid fa-stop', prefix='fa')

map = folium.Map(location=WHITEHOUSE_COORDS, zoom_start=7)
folium.Marker(WHITEHOUSE_COORDS, popup='Washington, DC', icon=start_icon).add_to(map)
folium.Marker(CLEVELAND_COORDS, popup='Cleveland, OH', icon=end_icon).add_to(map)
map

In [201]:
from evlens.visualization.folium_tools import plot_route

plot_route(WHITEHOUSE_COORDS, CLEVELAND_COORDS, 'White House', 'Cleveland, OH', route_coordinates=r, include_zoom_widget=False, starting_zoom=6)

## Find stations near route

This is the simple version that uses NREL's nearest-to-route API. We'll eventually use our own approach and our own station data of course.

In [ ]:
#TODO: have route builder return Route object as well as reversed coords for folium purposes (so you can generate the LineString here efficiently)


In [192]:
# Find NREL-supported charging stations near route
from evlens.data.nrel_api import AFDCStationsByRoute
from shapely.geometry import LineString

# Get WKT-structured route
route_linestring_wkt = str(LineString(route['features'][0]['geometry']['coordinates']))

afdc = AFDCStationsByRoute()
df = afdc.get_stations_near_route(route_linestring_wkt, 0.25, ev_connector_type='J1772COMBO') # 0.25 miles from initial route
# df = df[df['ev_network'] != 'Tesla']
df.info()

2024-07-30_T13_58_41EDT: INFO (evlens.data.nrel_api:L190) - 38 total records found, comprised of 97 plugs


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 75 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   access_code                 38 non-null     object 
 1   access_days_time            38 non-null     object 
 2   access_detail_code          3 non-null      object 
 3   cards_accepted              11 non-null     object 
 4   date_last_confirmed         38 non-null     object 
 5   expected_date               0 non-null      object 
 6   fuel_type_code              38 non-null     object 
 7   groups_with_access_code     38 non-null     object 
 8   id                          38 non-null     int64  
 9   maximum_vehicle_class       8 non-null      object 
 10  open_date                   38 non-null     object 
 11  owner_type_code             8 non-null      object 
 12  restricted_access           8 non-null      object 
 13  status_code                 38 non-nu

In [193]:
df['ev_network'].value_counts()

ev_network
ChargePoint Network    10
Non-Networked           7
eVgo Network            6
RIVIAN_ADVENTURE        6
Electrify America       5
SHELL_RECHARGE          2
Blink Network           1
Tesla                   1
Name: count, dtype: int64

In [194]:
from evlens.visualization.folium_tools import plot_route, add_stations_to_map

map = plot_route(WHITEHOUSE_COORDS, BOSTON_COORDS, 'White House', 'Boston, MA', route_coordinates=routes_coords, include_zoom_widget=False, starting_zoom=6)
map = add_stations_to_map(df, map, tooltip=True, networks_to_include='all')
map

2024-07-30_T13_58_48EDT: INFO (evlens.visualization.folium_tools:L138) - Counts of networks along the route: ev_network
ChargePoint Network    10
Non-Networked           7
eVgo Network            6
RIVIAN_ADVENTURE        6
Electrify America       5
SHELL_RECHARGE          2
Blink Network           1
Tesla                   1
Name: count, dtype: int64
